In [1]:
import h5py
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
sns.set()

In [2]:
# import the necessary keras packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from loader_util.preprocessing import ImageToArrayPreprocessor, AspectAwarePreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
##
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from imutils import paths

In [3]:
# import scoring and utility functions from sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, ShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, make_scorer
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs
#
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [4]:
# construct the argument parser
data_dir = r"C:\Users\mhasa\Google Drive\Tutorial Corner\PYTH\DeepLearning\DeepLearning-DL4CV\ImageDatasets\all_cats_dogs"

args = {
    "dataset": r"{data_dir}//hdf5//extracted_features.hdf5",
    "model": fr"{data_dir}//outputs//extracted_features_lr_model.pt",
    "jobs": -1
}

In [ ]:
# open the hdf5 dataset to get the extracted features
db = h5py.File(args["dataset"], "r")
i = int(db["labels"].shape[0] * 0.75)

In [ ]:
# define the set of gridsearch parameters
print(f'[INFO] tuning hyperparameters.....')
params = {"C": [0.0001, 0.001, 0.01, 0.1, 1.0]}
model = GridSearchCV(LogisticRegression(solver="lbfgs",
                                        multi_class="auto"),
                     param_grid=params,
                     cv=3,
                     n_jobs=args["jobs"])
model.fit(db["extracted_features"][:i], db["labels"][:i])
print(f'[INFO] best hyperparameters are: {model.best_params_}.....')

In [ ]:
print(f'[INFO] evaluating.....')
preds = model.predict(db["extracted_features"][i:])
print(classification_report(db["labels"][i:], preds,
                            target_names=db["label_names"]))

# compute the raw accuracy with extra precision
acc = accuracy_score(db["labels"][i:], preds)
print(f'[INFO] score: {acc}.....')

In [ ]:
# serialise the model to disk
print(f'[INFO] saving model.....')
f = open(args["model"], "wb")
f.write(pickle.dumps(model.best_estimator_))
f.close()
db.close()